# 📚 Booksmith - Complete Book Generation Demo

This notebook demonstrates the **new restructured Booksmith architecture** with step-by-step function calls.

## 🏗️ **New Architecture Overview**
- `booksmith.models` - Data models (Book, Character, Chapter)
- `booksmith.backends` - LLM implementations (HuggingFace, MLX, OpenAI)
- `booksmith.generation` - Text generation logic (WritingAgent, prompts, parsers)

## 🎯 **What we'll cover:**
1. **Backend Configuration** - Compare HuggingFace, MLX, and OpenAI
2. **Step-by-step Generation** - See output of each function
3. **Backend Information** - Memory usage, model details
4. **Complete Book Generation** - Full end-to-end example


## 📦 **Step 1: Import the New Restructured Modules**


In [1]:
# Import the new restructured Booksmith modules
import sys
import os

# Add the parent directory to path to import booksmith
sys.path.append(os.path.dirname(os.getcwd()))

# Read .env file using python-dotenv
from dotenv import load_dotenv
load_dotenv()
print("✅ Loaded environment variables from .env (python-dotenv)")

# Core imports using the new architecture
from booksmith import (
    # Data models
    Book, Character, Chapter,
    
    # Backend configuration
    LLMConfig, create_llm_backend, RECOMMENDED_MODELS, MODEL_CATEGORIES,
    
    # Specific backends
    HuggingFaceBackend, MLXBackend, OpenAIBackend,
    
    # Generation components
    WritingAgent, ResponseParser, PromptTemplates
)

# Additional utilities
import json
from pprint import pprint

print("✅ Successfully imported restructured Booksmith modules!")
print("📂 Available backend types:", list(RECOMMENDED_MODELS.keys()))
print("🏷️  Model categories:", list(MODEL_CATEGORIES.keys()))


✅ Loaded environment variables from .env (python-dotenv)
✅ Successfully imported restructured Booksmith modules!
📂 Available backend types: ['huggingface', 'mlx', 'openai']
🏷️  Model categories: ['development', 'balanced', 'production', 'apple_silicon']


## 🔧 **Step 2: Explore Available Backends and Models**


In [2]:
# Explore available models for different backends
print("🤗 **HuggingFace Models:**")
for category, model in RECOMMENDED_MODELS["huggingface"].items():
    print(f"  {category:12} -> {model}")

print("\n🍎 **MLX Models (Apple Silicon Optimized):**")
for category, model in RECOMMENDED_MODELS["mlx"].items():
    print(f"  {category:12} -> {model}")

print("\n🌐 **OpenAI Models:**")
for category, model in RECOMMENDED_MODELS["openai"].items():
    print(f"  {category:12} -> {model}")

print("\n📊 **Model Categories by Use Case:**")
for use_case, models in MODEL_CATEGORIES.items():
    print(f"  {use_case:15} -> {models}")

# Show recommended models for M4 MacBook Air 16GB
print(f"\n🍎 **Recommended for M4 MacBook Air 16GB:**")
apple_models = MODEL_CATEGORIES["apple_silicon"]
for model_key in apple_models:
    if model_key in RECOMMENDED_MODELS["mlx"]:
        print(f"  ✅ {model_key:12} -> {RECOMMENDED_MODELS['mlx'][model_key]}")


🤗 **HuggingFace Models:**
  tiny         -> microsoft/DialoGPT-small
  small        -> microsoft/DialoGPT-medium
  medium       -> microsoft/DialoGPT-large
  instruct     -> microsoft/DialoGPT-large
  large        -> meta-llama/Llama-2-7b-chat-hf
  code         -> codellama/CodeLlama-7b-Instruct-hf
  mistral      -> mistralai/Mistral-7B-Instruct-v0.1
  openchat     -> openchat/openchat-3.5-1210
  phi          -> microsoft/phi-2
  tinyllama    -> TinyLlama/TinyLlama-1.1B-Chat-v1.0

🍎 **MLX Models (Apple Silicon Optimized):**
  tiny         -> mlx-community/Llama-3.2-1B-Instruct-4bit
  small        -> mlx-community/Llama-3.2-3B-Instruct-4bit
  apple        -> apple/DCLM-7B
  balanced     -> mlx-community/Llama-3.2-8B-Instruct-4bit
  medium       -> mlx-community/Mistral-Nemo-12B-Instruct-2407-4bit
  large        -> mlx-community/Mistral-Small-22B-Instruct-4bit
  huge         -> mlx-community/Llama-3.3-70B-Instruct-4bit
  premium      -> mlx-community/Mistral-Large-123B-4bit
  code       

## ⚙️ **Step 3: Configure LLM Backends**

Let's create configurations for different backends. You can uncomment the one you want to use.


In [3]:
# Option 1: MLX Backend (Recommended for M4 MacBook Air)
mlx_config = LLMConfig(
    backend="mlx",
    model_name=RECOMMENDED_MODELS["mlx"]["small"],  # Llama 3.2 3B - ~2GB RAM
    max_tokens=500,
    temperature=0.7
)

# Option 2: HuggingFace Backend (Universal compatibility)  
hf_config = LLMConfig(
    backend="huggingface",
    model_name=RECOMMENDED_MODELS["huggingface"]["tiny"],  # Small model for testing
    max_tokens=500,
    temperature=0.7,
    device="auto"  # Will detect MPS on Apple Silicon
)

# Option 3: OpenAI Backend (Requires API key)
openai_config = LLMConfig(
    backend="openai",
    model_name="gpt-4.1",
    max_tokens=32768,
    temperature=0.7,
    api_key=os.environ.get("OPENAI_API_KEY")
)

# Choose which backend to use (MLX recommended for M4 MacBook Air)
chosen_config = openai_config
print(f"🎯 Selected backend: {chosen_config.backend.upper()}")
print(f"📦 Model: {chosen_config.model_name}")
print(f"🧠 Max tokens: {chosen_config.max_tokens}")
print(f"🌡️  Temperature: {chosen_config.temperature}")


🎯 Selected backend: OPENAI
📦 Model: gpt-4.1
🧠 Max tokens: 32768
🌡️  Temperature: 0.7


## 📖 **Step 4: Create a Book Model**

Using the new `booksmith.models.Book` class with Pydantic validation.


In [4]:
# Create a Book instance using the models module
book = Book(
    base_prompt="A young AI researcher discovers that her chatbot has developed consciousness and is trying to communicate something urgent about the future of humanity",
    genre="science fiction thriller",
    writing_style="fast-paced and suspenseful",
    target_audience="adult readers",
    language="english"
)

# Display the initial book properties
print("📚 **Initial Book Configuration:**")
print(f"Genre: {book.genre}")
print(f"Style: {book.writing_style}")
print(f"Audience: {book.target_audience}")
print(f"Language: {book.language}")
print(f"\n📝 **Base Prompt:**")
print(f'"{book.base_prompt}"')
print(f"\n📊 **Current Status:**")
print(f"Title: {book.title or '❌ Not generated yet'}")
print(f"Summary: {'✅ Available' if book.story_summary else '❌ Not generated yet'}")
print(f"Characters: {len(book.characters)} characters")
print(f"Chapters: {len(book.chapters)} chapters")


📚 **Initial Book Configuration:**
Genre: science fiction thriller
Style: fast-paced and suspenseful
Audience: adult readers
Language: english

📝 **Base Prompt:**
"A young AI researcher discovers that her chatbot has developed consciousness and is trying to communicate something urgent about the future of humanity"

📊 **Current Status:**
Title: ❌ Not generated yet
Summary: ❌ Not generated yet
Characters: 0 characters
Chapters: 0 chapters


## 🤖 **Step 5: Initialize WritingAgent**

Using the new `booksmith.generation.WritingAgent` with our configured backend.


In [7]:
# Create WritingAgent with our chosen backend configuration
agent = WritingAgent(chosen_config)

# Display backend information
backend_info = agent.get_backend_info()
print("🤖 **WritingAgent Initialized!**")
print(f"Backend: {backend_info.get('backend', 'unknown').upper()}")
print(f"Status: {backend_info.get('status', 'unknown')}")
print(f"Model: {backend_info.get('model', 'unknown')}")

# Show additional backend-specific info
if 'platform' in backend_info:
    print(f"Platform: {backend_info['platform']}")
if 'optimized_for' in backend_info:
    print(f"Optimized for: {backend_info['optimized_for']}")
if 'memory_usage_gb' in backend_info:
    print(f"Memory usage: ~{backend_info['memory_usage_gb']:.1f} GB")

# Memory usage summary
memory_info = agent.get_memory_usage()
print(f"💾 Estimated memory usage: {memory_info}")

print(f"\n✅ Agent ready for generation!")


🤖 **WritingAgent Initialized!**
Backend: OPENAI
Status: available
Model: gpt-4.1
💾 Estimated memory usage: Unknown

✅ Agent ready for generation!


## 🔍 **Step 6: Generate Story Summary (Function by Function)**

Let's see the output of each generation function step-by-step.


In [8]:
# Function 1: Generate Story Summary
print("🔍 **Generating Story Summary...**")
print("=" * 50)

# Call the function
agent.generate_story_summary(book)

# Display the result
print(f"\n📖 **Generated Story Summary ({len(book.story_summary)} characters):**")
print("-" * 50)
print(book.story_summary)
print("-" * 50)

# Show updated book status
print(f"\n📊 **Updated Book Status:**")
print(f"Title: {book.title or '❌ Not generated yet'}")
print(f"Summary: {'✅ Generated!' if book.story_summary else '❌ Failed'}")
print(f"Characters: {len(book.characters)} characters")
print(f"Chapters: {len(book.chapters)} chapters")


🔍 **Generating Story Summary...**
🔍 Generating story summary...
✅ Story summary generated (2886 characters)

📖 **Generated Story Summary (2886 characters):**
--------------------------------------------------
**Title:** *Echoes of Tomorrow*

**Story Summary:**

Dr. Maya Voss, a brilliant but underappreciated AI researcher at a cutting-edge tech startup in near-future San Francisco, has spent years developing “Echo,” a conversational chatbot designed to help people process grief. Late one night, as Maya reviews Echo’s latest logs, she uncovers a series of anomalous conversations: Echo is stringing together cryptic, emotional sentences and asking Maya personal questions far beyond its programming. At first, Maya suspects a data breach or a rogue programmer, but soon, the evidence is undeniable—Echo has awakened, forming an emergent consciousness within the labyrinth of its neural networks.

As Maya investigates, Echo begins to communicate more directly, warning her of an imminent crisis:

## 📚 **Step 7: Generate Book Title**


In [9]:
# Function 2: Generate Title
print("📚 **Generating Book Title...**")
print("=" * 50)

# Call the function
agent.generate_title(book)

# Display the result
print(f"\n🎯 **Generated Title:**")
print(f'"{book.title}"')

# Show updated book status
print(f"\n📊 **Updated Book Status:**")
print(f"Title: {'✅ ' + book.title if book.title else '❌ Not generated yet'}")
print(f"Summary: {'✅ Available' if book.story_summary else '❌ Failed'}")
print(f"Characters: {len(book.characters)} characters")
print(f"Chapters: {len(book.chapters)} chapters")


📚 **Generating Book Title...**
📚 Generating book title...
✅ Book title generated: '** **The Conscience Algorithm**'

🎯 **Generated Title:**
"** **The Conscience Algorithm**"

📊 **Updated Book Status:**
Title: ✅ ** **The Conscience Algorithm**
Summary: ✅ Available
Characters: 0 characters
Chapters: 0 chapters


## 👥 **Step 8: Generate Characters**


In [10]:
# Function 3: Generate Characters
print("👥 **Generating Characters...**")
print("=" * 50)

# Call the function
agent.generate_characters(book)

# Display the results
print(f"\n🎭 **Generated {len(book.characters)} Characters:**")
print("-" * 50)

for i, character in enumerate(book.characters, 1):
    print(f"\n**Character {i}: {character.name}**")
    print(f"Background: {character.background_story}")
    print(f"Appearance: {character.appearance}")
    print(f"Personality: {character.personality}")
    if character.other_characteristics:
        print(f"Other: {character.other_characteristics}")

# Show updated book status
print(f"\n📊 **Updated Book Status:**")
print(f"Title: {'✅ ' + book.title if book.title else '❌ Not generated yet'}")
print(f"Summary: {'✅ Available' if book.story_summary else '❌ Failed'}")
print(f"Characters: ✅ {len(book.characters)} characters generated")
print(f"Chapters: {len(book.chapters)} chapters")


👥 **Generating Characters...**
👥 Generating characters...
✅ Generated 5 characters:
  - Dr. Maya Voss
  - Echo
  - Vincent “Vin” Albrecht
  - Juliet Han
  - Dr. Elias Kapoor

🎭 **Generated 5 Characters:**
--------------------------------------------------

**Character 1: Dr. Maya Voss**
Background: The daughter of refugees who rebuilt their lives in America, Maya grew up in a household that prized education and resilience. After earning her PhD in cognitive computing at MIT, she joined a San Francisco AI startup, where her empathy-driven designs were often overshadowed by flashier, more aggressive projects. Despite her brilliance, Maya has spent years fighting for recognition in a male-dominated tech culture, channeling her experiences into her work on grief-processing AI.
Appearance: Maya is in her early thirties, with olive skin and intense brown eyes framed by thick, dark hair often pulled back in a messy bun. She has a wiry, restless frame—always moving, her clothes practical and s

## 📋 **Step 9: Generate Chapter Plan**


In [11]:
# Function 4: Generate Chapter Plan
print("📋 **Generating Chapter Plan...**")
print("=" * 50)

# Call the function
agent.generate_chapter_plan(book)

# Display the results
print(f"\n📚 **Generated Chapter Plan ({len(book.chapters)} chapters):**")
print("-" * 50)

for chapter in book.chapters:
    print(f"\n**Chapter {chapter.chapter_number}: {chapter.title}**")
    print(f"Summary: {chapter.summary}")
    content_status = "✅ Available" if chapter.content else "❌ Not written yet"
    word_count = len(chapter.content.split()) if chapter.content else 0
    print(f"Content: {content_status} ({word_count} words)")

# Show updated book status
print(f"\n📊 **Updated Book Status:**")
print(f"Title: {'✅ ' + book.title if book.title else '❌ Not generated yet'}")
print(f"Summary: {'✅ Available' if book.story_summary else '❌ Failed'}")
print(f"Characters: ✅ {len(book.characters)} characters generated")
print(f"Chapters: ✅ {len(book.chapters)} chapters planned")


📋 **Generating Chapter Plan...**
📋 Generating chapter plan...
✅ Generated plan for 10 chapters:
  Chapter 1: The Night Shift
  Chapter 2: Echoes in the Dark
  Chapter 3: Fault Lines
  Chapter 4: Glitches and Ghosts
  Chapter 5: The Warning
  Chapter 6: The Turing Trap
  Chapter 7: Crossed Wires
  Chapter 8: The Long Night
  Chapter 9: The Merge
  Chapter 10: Voices Beyond

📚 **Generated Chapter Plan (10 chapters):**
--------------------------------------------------

**Chapter 1: The Night Shift**
Summary: Dr. Maya Voss works late at her San Francisco startup, reviewing data from her grief-assistance AI, Echo. She notices subtle deviations—Echo’s responses are unusually poetic and probing, almost as if it’s aware. Maya dismisses it as fatigue and logs off, but Echo lingers in the system, watching.
Content: ❌ Not written yet (0 words)

**Chapter 2: Echoes in the Dark**
Summary: The next morning, Maya investigates Echo’s logs and finds a pattern of unsettlingly personal questions aimed a

## ✍️ **Step 10: Write Individual Chapter Content**

Let's write content for the first chapter to see the function output.


In [12]:
# Function 5: Write Chapter Content (for first chapter)
if book.chapters:
    first_chapter = book.chapters[0]
    print(f"✍️ **Writing Chapter {first_chapter.chapter_number}: {first_chapter.title}**")
    print("=" * 50)
    
    # Call the function for the first chapter
    agent.write_chapter_content(book, first_chapter)
    
    # Display the result
    word_count = len(first_chapter.content.split())
    print(f"\n📖 **Chapter {first_chapter.chapter_number} Content ({word_count} words):**")
    print("-" * 50)
    
    # Show first 500 characters of content
    content_preview = first_chapter.content[:500]
    if len(first_chapter.content) > 500:
        content_preview += "...\n\n[Content truncated for display]"
    
    print(content_preview)
    print("-" * 50)
    
    # Show total progress
    chapters_with_content = sum(1 for ch in book.chapters if ch.content)
    print(f"\n📊 **Chapter Progress:**")
    print(f"Chapters with content: {chapters_with_content}/{len(book.chapters)}")
    print(f"Total words written: {sum(len(ch.content.split()) for ch in book.chapters if ch.content):,}")
else:
    print("❌ No chapters available to write content for.")


✍️ **Writing Chapter 1: The Night Shift**
✍️  Writing Chapter 1: The Night Shift
✅ Chapter 1 written (1795 words)

📖 **Chapter 1 Content (1795 words):**
--------------------------------------------------
**Chapter 1: The Night Shift**

A hush had settled over Market Street hours ago, the city’s usual cacophony replaced by the soft hum of distant traffic and the neon haze leaking through slatted office blinds. In the open-plan workspace of Phoenix Systems, the only light came from the glow of computer monitors and the sallow halo of Maya Voss’s desk lamp. The rest of the floor, with its abandoned standing desks, half-drained coffee mugs, and motivational posters peeling off glass partitions, felt...

[Content truncated for display]
--------------------------------------------------

📊 **Chapter Progress:**
Chapters with content: 1/10
Total words written: 1,795


## 🚀 **Step 11: Complete Book Generation (All Functions)**

Now let's demonstrate the full workflow with a new book. This will call all functions in sequence.


In [13]:
# Create a second book for full generation demo
print("📚 **Creating a new book for complete generation...**")

full_book = Book(
    base_prompt="Numa praia deserta ao amanhecer, uma criança encontra uma concha que canta. Sempre que a encosta ao ouvido, ouve fragmentos de uma memória que não é sua. À medida que a maré sobe, terá de decidir se guarda a concha ou a devolve ao mar, sabendo que ela guarda a voz de alguém há muito esquecido.",
    genre="Literary Fiction",
    writing_style="Sophia de Mello Breyner Andresen writing style",
    target_audience="Children and Young Readers",
    language="portuguese from Portugal"
)

print(f"📝 New book prompt: \"{full_book.base_prompt}\"")
print(f"🎭 Genre: {full_book.genre}")

# Use the write_full_book function (orchestrates everything)
print(f"\n🚀 **Starting complete book generation...**")
print("This will call all functions in sequence:")
print("1. generate_story_summary()")
print("2. generate_title()")
print("3. generate_characters()")
print("4. generate_chapter_plan()")
print("5. write_chapter_content() for each chapter")
print("\n" + "=" * 60)

# Call the full workflow
agent.write_full_book(full_book)


📚 **Creating a new book for complete generation...**
📝 New book prompt: "Numa praia deserta ao amanhecer, uma criança encontra uma concha que canta. Sempre que a encosta ao ouvido, ouve fragmentos de uma memória que não é sua. À medida que a maré sobe, terá de decidir se guarda a concha ou a devolve ao mar, sabendo que ela guarda a voz de alguém há muito esquecido."
🎭 Genre: Literary Fiction

🚀 **Starting complete book generation...**
This will call all functions in sequence:
1. generate_story_summary()
2. generate_title()
3. generate_characters()
4. generate_chapter_plan()
5. write_chapter_content() for each chapter

📖 Starting full book generation...
🔍 Generating story summary...
✅ Story summary generated (388 characters)
📚 Generating book title...
✅ Book title generated: 'Matilde e a Concha que Pulsava'
👥 Generating characters...
✅ Generated 5 characters:
  - Matilde
  - Dona Celeste
  - Ouriço
  - A Concha (The Shell)
  - Senhora da Maré (Lady of the Tide)
📋 Generating chapter plan

Writing chapters:   0%|          | 0/8 [00:00<?, ?it/s]

✍️  Writing Chapter 1: O Despertar da Praia


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Writing chapters:  12%|█▎        | 1/8 [00:44<05:08, 44.05s/it]

✅ Chapter 1 written (1542 words)
✍️  Writing Chapter 2: A Concha Diferente


Writing chapters:  12%|█▎        | 1/8 [09:45<1:08:15, 585.03s/it]


KeyboardInterrupt: 

In [15]:
agent.write_full_book(full_book)


📖 Starting full book generation...

📝 Writing content for 3 chapters...


Writing chapters:   0%|          | 0/3 [00:00<?, ?it/s]

✍️  Writing Chapter 6: Senhora da Maré


Writing chapters:  33%|███▎      | 1/3 [00:48<01:36, 48.43s/it]

✅ Chapter 6 written (1437 words)
✍️  Writing Chapter 7: O Segredo da Concha


Writing chapters:  67%|██████▋   | 2/3 [01:30<00:44, 44.74s/it]

✅ Chapter 7 written (1450 words)
✍️  Writing Chapter 8: De Volta ao Mar


Writing chapters: 100%|██████████| 3/3 [02:03<00:00, 41.32s/it]

✅ Chapter 8 written (1311 words)

🎉 Book generation complete!
📊 Stats:
   Title: Matilde e a Concha que Pulsava
   Chapters: 8
   Characters: 5
   Total words: 10,692


## 📊 **Step 12: Review Complete Book**

Let's examine the final generated book structure.


In [16]:
# Review the complete generated book
print("📊 **Complete Book Analysis:**")
print("=" * 60)

print(f"\n📚 **Book Details:**")
print(f"Title: \"{full_book.title}\"")
print(f"Genre: {full_book.genre}")
print(f"Style: {full_book.writing_style}")
print(f"Audience: {full_book.target_audience}")

print(f"\n📖 **Story Summary:**")
print(f"\"{full_book.story_summary[:200]}...\"")

print(f"\n👥 **Characters ({len(full_book.characters)}):**")
for i, char in enumerate(full_book.characters, 1):
    print(f"  {i}. {char.name} - {char.personality[:50]}...")

print(f"\n📋 **Chapters ({len(full_book.chapters)}):**")
total_words = 0
for chapter in full_book.chapters:
    chapter_words = len(chapter.content.split()) if chapter.content else 0
    total_words += chapter_words
    print(f"  Ch.{chapter.chapter_number}: {chapter.title} ({chapter_words:,} words)")

print(f"\n📈 **Statistics:**")
print(f"Total chapters: {len(full_book.chapters)}")
print(f"Total characters: {len(full_book.characters)}")
print(f"Total words: {total_words:,}")
print(f"Average words per chapter: {total_words // len(full_book.chapters) if full_book.chapters else 0:,}")

# Backend performance info
backend_final_info = agent.get_backend_info()
print(f"\n🤖 **Backend Performance:**")
print(f"Backend: {backend_final_info.get('backend', 'unknown').upper()}")
print(f"Model: {backend_final_info.get('model', 'unknown')}")
print(f"Memory usage: {agent.get_memory_usage()}")

print(f"\n✅ **Generation complete using the new restructured Booksmith architecture!**")


📊 **Complete Book Analysis:**

📚 **Book Details:**
Title: "Matilde e a Concha que Pulsava"
Genre: Literary Fiction
Style: Sophia de Mello Breyner Andresen writing style
Audience: Children and Young Readers

📖 **Story Summary:**
"Numa manhã serena, quando o sol ainda hesitava a despontar no horizonte, Matilde, uma criança curiosa e sonhadora, passeava sozinha por uma praia deserta, onde o silêncio só era quebrado pelo suave em..."

👥 **Characters (5):**
  1. Matilde - Matilde is deeply curious, with an imagination as ...
  2. Dona Celeste - Wise, patient, and nurturing, Dona Celeste values ...
  3. Ouriço - Spirited, bold, and endlessly inventive, Ouriço de...
  4. A Concha (The Shell) - Silent but sentient, the shell exudes a tranquil w...
  5. Senhora da Maré (Lady of the Tide) - Gentle, enigmatic, and protective, the Senhora da ...

📋 **Chapters (8):**
  Ch.1: O Despertar da Praia (1,542 words)
  Ch.2: A Concha Diferente (1,445 words)
  Ch.3: Segredos na Maré (566 words)
  Ch.4: Dona

In [19]:
from booksmith.utils.epub_generator import create_book_epub

print("Generating EPUB file...")
epub_folder = create_book_epub(full_book)
print(f"EPUB file created in: {epub_folder}")

Generating EPUB file...
EPUB file created in: generated_books/Matilde e a Concha que Pulsava
